<a href="https://colab.research.google.com/github/explorewtxs/AI_pai_second_test/blob/main/step2/fine_tune/Fine_tune_on_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install scikit-learn
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
from datasets import load_dataset
data = load_dataset("stanfordnlp/imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
small_train_dataset = data["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = data["test"].shuffle(seed=42).select(range(1000))
data
data["train"][1000]

{'text': 'Although I have to admit I laughed more watching this movie than the last few comedies I saw.<br /><br />The budget must have consisted of pocket change from the actors. The production values are so low that they actual made it kind of fun to watch. Reminds me of the Robot Monster made up of a guy in a gorilla suit with a cardboard diving helmet on.<br /><br />In one scene a hapless victim gets their arm and leg cut off. Geez, hard to believe but the Black Knight scene from Holy Grail was more realistic. I kept wondering why the victim didn\'t start shouting " None Shall Pass" and " It\'s only a flesh wound, I\'ve had worse". It was one of the funniest scenes I\'ve seen in the past year.<br /><br />The "gladiator/demon" was a stitch too. Between the horribly cheap costume and the geeky look of the guy in it the end result was hysterical.<br /><br />Truly a movie that is bad enough to be watchable. Kind of like seeing a slow motion auto accident on film.<br /><br />',
 'label'

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
tokenizer.pad_token = tokenizer.eos_token# GPT-2的tokenizer没有padding token，设置它为eos token
def tokenize_function(examples):
  return tokenizer(examples["text"],padding = "max_length",truncation = True)
Train = small_train_dataset.map(tokenize_function,batched=True)
Eval = small_eval_dataset.map(tokenize_function,batched=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [5]:
import numpy as np
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
# 定义模型评价函数
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  preds = np.argmax(logits, axis=-1)
  '''for i in range(100):
    print("show_detail")
    print(eval_pred[0][i])
    print(labels[i])
    print(preds[i])'''
  precision,recall,f1,_ = precision_recall_fscore_support(labels,preds,average='binary')#先label后preds，不要弄反了
  acc = accuracy_score(labels,preds)
  return {
      'accuracy':acc,
      'f1':f1,
      'precision':precision,
      'recall':recall
  }

In [6]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("openai-community/gpt2",num_labels = 2)
model.config.pad_token_id = model.config.eos_token_id# 将模型的pad token id设置为eos token id


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir = 'result',
    eval_strategy = "epoch",
    num_train_epochs = 3,
    learning_rate = 1e-4,
    weight_decay = 0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=Train,
    eval_dataset=Eval,
    compute_metrics=compute_metrics,
)


In [8]:
results = trainer.evaluate()
print(f"Accuracy: {results['eval_accuracy']}")
print(f"F1-Score: {results['eval_f1']}")
print(f"Precision: {results['eval_precision']}")
print(f"Recall: {results['eval_recall']}")

Accuracy: 0.51
F1-Score: 0.012096774193548387
Precision: 0.375
Recall: 0.006147540983606557


In [9]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy,F1,Precision,Recall
1,No log,0.321990,0.002700,0.879000,0.871686,0.903297,0.842213
2,No log,0.389268,0.002700,0.875000,0.879227,0.831810,0.932377
3,No log,0.534232,0.002700,0.893000,0.892678,0.874263,0.911885


TrainOutput(global_step=375, training_loss=0.3119270222981771, metrics={'train_runtime': 946.8391, 'train_samples_per_second': 3.168, 'train_steps_per_second': 0.396, 'total_flos': 1567780503552000.0, 'train_loss': 0.3119270222981771, 'epoch': 3.0})

In [10]:
results = trainer.evaluate()
print(f"Accuracy: {results['eval_accuracy']}")
print(f"F1-Score: {results['eval_f1']}")
print(f"Precision: {results['eval_precision']}")
print(f"Recall: {results['eval_recall']}")


Accuracy: 0.893
F1-Score: 0.8926780341023068
Precision: 0.8742632612966601
Recall: 0.9118852459016393
